In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
nn_aqi_df = pd.read_csv("AirQualityAndWeatherDataMerged.csv")
nn_aqi_df.head()

,Unnamed: 0,state,county,city,date_local,parameter,sample_duration,units_of_measure,arithmetic_mean,aqi,...,Precipitation,Snow,Snow Depth,Wind Speed,Wind Direction,Wind Gust,Visibility,Cloud Cover,Relative Humidity,Conditions
0,0,Tennessee,Sumner,Hendersonville,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),6.2,26.0,...,0.00,0.0,0.0,13.4,292.13,19.7,9.9,32.2,58.99,Partially cloudy
1,1,Tennessee,Davidson,Nashville,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),5.1,21.0,...,0.00,0.0,0.0,13.9,278.04,21.9,9.9,32.1,57.18,Partially cloudy
2,2,Tennessee,Davidson,Nashville,2018-01-02,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),3.8,16.0,...,0.00,0.0,0.0,8.8,153.88,NaN,9.9,24.8,50.71,Clear
3,3,Tennessee,Sumner,Hendersonville,2018-01-02,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),5.6,23.0,...,0.00,0.0,0.0,7.9,166.08,NaN,9.9,18.0,52.93,Clear
4,4,Tennessee,Sumner,Hendersonville,2018-01-03,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),5.3,22.0,...,0.01,0.0,0.0,11.9,252.83,24.2,9.9,60.1,50.06,"Rain, Partially cloudy"


In [2]:
# Binning aqi values
aqi_bins = [-1, 50, 100, 300, 500]
group_names = ['Good', 'Moderate', 'Unhealthy', 'Hazardous']

# Categorize aqi based on the bins.
nn_aqi_df["AQI_Range"] = pd.cut(nn_aqi_df["aqi"], aqi_bins, labels=group_names)

nn_aqi_df

,Unnamed: 0,state,county,city,date_local,parameter,sample_duration,units_of_measure,arithmetic_mean,aqi,...,Snow,Snow Depth,Wind Speed,Wind Direction,Wind Gust,Visibility,Cloud Cover,Relative Humidity,Conditions,AQI_Range
0,0,Tennessee,Sumner,Hendersonville,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),6.2,26.0,...,0.0,0.0,13.4,292.13,19.7,9.9,32.2,58.99,Partially cloudy,Good
1,1,Tennessee,Davidson,Nashville,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),5.1,21.0,...,0.0,0.0,13.9,278.04,21.9,9.9,32.1,57.18,Partially cloudy,Good
2,2,Tennessee,Davidson,Nashville,2018-01-02,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),3.8,16.0,...,0.0,0.0,8.8,153.88,NaN,9.9,24.8,50.71,Clear,Good
3,3,Tennessee,Sumner,Hendersonville,2018-01-02,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),5.6,23.0,...,0.0,0.0,7.9,166.08,NaN,9.9,18.0,52.93,Clear,Good
4,4,Tennessee,Sumner,Hendersonville,2018-01-03,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),5.3,22.0,...,0.0,0.0,11.9,252.83,24.2,9.9,60.1,50.06,"Rain, Partially cloudy",Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,1667,Tennessee,Davidson,Nashville,2020-12-30,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),8.7,36.0,...,0.0,0.0,23.8,170.63,38.1,9.3,68.3,63.09,"Rain, Partially cloudy",Good
1668,1668,Tennessee,Sumner,Hendersonville,2020-12-30,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),7.3,30.0,...,0.0,0.0,22.1,185.75,36.1,9.3,67.1,64.16,"Rain, Partially cloudy",Good
1669,1669,Tennessee,Sumner,Hendersonville,2020-12-31,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),1.0,4.0,...,0.0,0.0,16.8,127.13,21.8,7.7,97.0,91.89,"Rain, Overcast",Good
1670,1670,Tennessee,Davidson,Nashville,2020-12-31,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),2.2,9.0,...,0.0,0.0,17.4,128.83,25.0,7.6,98.8,90.33,"Rain, Overcast",Good


In [3]:
# Checking value counts
nn_aqi_df["AQI_Range"].value_counts()

Good         1503
Moderate      167
Unhealthy       2
Hazardous       0
Name: AQI_Range, dtype: int64

In [4]:
# Converting text to code value for AQI_Range
nn_aqi_df["AQI_Range"] = nn_aqi_df["AQI_Range"].astype('category').cat.codes

In [5]:
# Checking value counts
nn_aqi_df["AQI_Range"].value_counts()

0    1503
1     167
2       2
Name: AQI_Range, dtype: int64

In [6]:
nn_aqi_df.dtypes

Unnamed: 0               int64
state                   object
county                  object
city                    object
date_local              object
parameter               object
sample_duration         object
units_of_measure        object
arithmetic_mean        float64
aqi                    float64
City                    object
State                   object
Country                 object
Date time               object
Maximum Temperature    float64
Minimum Temperature    float64
Temperature            float64
Wind Chill             float64
Heat Index             float64
Precipitation          float64
Snow                   float64
Snow Depth             float64
Wind Speed             float64
Wind Direction         float64
Wind Gust              float64
Visibility             float64
Cloud Cover            float64
Relative Humidity      float64
Conditions              object
AQI_Range                 int8
dtype: object

In [7]:
# Generate our categorical variable lists
nn_aqi_cat = nn_aqi_df.dtypes[nn_aqi_df.dtypes == "object"].index.tolist()
nn_aqi_cat

['state',
 'county',
 'city',
 'date_local',
 'parameter',
 'sample_duration',
 'units_of_measure',
 'City',
 'State',
 'Country',
 'Date time',
 'Conditions']

In [8]:
# Drop the non-beneficial columns, "Unnamed: 0", "parameter", "sample_duration", "units_of_measure", "state", "county", "Date time", "City", "State", "Country", "arithmetic_mean", "date_local", "city", "Maximum Temperature", "Minimum Temperature", "Wind Chill", "Snow Depth", "Wind Gust", "Heat Index", "Wind Direction", "Snow", "Visibility", "Cloud Cover", "aqi"
nn_aqi_df = nn_aqi_df.drop(columns=["Unnamed: 0", "parameter", "sample_duration", "units_of_measure", "state", "county", "Date time", "City", "State", "Country", "arithmetic_mean", "date_local", "city", "Maximum Temperature", "Minimum Temperature", "Wind Chill", "Snow Depth", "Wind Gust", "Heat Index", "Wind Direction", "Snow", "Visibility", "Cloud Cover", "aqi" ])
nn_aqi_df

,Temperature,Precipitation,Wind Speed,Relative Humidity,Conditions,AQI_Range
0,13.3,0.00,13.4,58.99,Partially cloudy,0
1,13.6,0.00,13.9,57.18,Partially cloudy,0
2,14.6,0.00,8.8,50.71,Clear,0
3,13.9,0.00,7.9,52.93,Clear,0
4,24.5,0.01,11.9,50.06,"Rain, Partially cloudy",0
...,...,...,...,...,...,...
1667,56.0,0.12,23.8,63.09,"Rain, Partially cloudy",0
1668,55.5,0.25,22.1,64.16,"Rain, Partially cloudy",0
1669,41.0,0.90,16.8,91.89,"Rain, Overcast",0
1670,41.6,0.86,17.4,90.33,"Rain, Overcast",0


In [9]:
# Determine the number of unique values in each column.
nn_aqi_df.nunique()

Temperature           536
Precipitation         136
Wind Speed            202
Relative Humidity    1295
Conditions              9
AQI_Range               3
dtype: int64

In [10]:
# Generate our categorical variable lists
application_cat = nn_aqi_df.dtypes[nn_aqi_df.dtypes == "object"].index.tolist()
application_cat

['Conditions']

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(nn_aqi_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,Conditions_Clear,Conditions_Overcast,Conditions_Partially cloudy,Conditions_Rain,"Conditions_Rain, Overcast","Conditions_Rain, Partially cloudy",Conditions_Snow,"Conditions_Snow, Overcast","Conditions_Snow, Partially cloudy"
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [12]:
# Merge one-hot encoded features and drop the originals
nn_aqi_df = nn_aqi_df.merge(encode_df, left_index=True,right_index=True)
nn_aqi_df = nn_aqi_df.drop(columns = application_cat)
nn_aqi_df.head()

,Temperature,Precipitation,Wind Speed,Relative Humidity,AQI_Range,Conditions_Clear,Conditions_Overcast,Conditions_Partially cloudy,Conditions_Rain,"Conditions_Rain, Overcast","Conditions_Rain, Partially cloudy",Conditions_Snow,"Conditions_Snow, Overcast","Conditions_Snow, Partially cloudy"
0,13.3,0.00,13.4,58.99,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.6,0.00,13.9,57.18,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14.6,0.00,8.8,50.71,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13.9,0.00,7.9,52.93,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,24.5,0.01,11.9,50.06,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [13]:
# Determine the number of unique values in each column.
nn_aqi_df.nunique()

Temperature                           536
Precipitation                         136
Wind Speed                            202
Relative Humidity                    1295
AQI_Range                               3
Conditions_Clear                        2
Conditions_Overcast                     2
Conditions_Partially cloudy             2
Conditions_Rain                         2
Conditions_Rain, Overcast               2
Conditions_Rain, Partially cloudy       2
Conditions_Snow                         2
Conditions_Snow, Overcast               2
Conditions_Snow, Partially cloudy       2
dtype: int64

In [14]:
# Split our preprocessed data into our features and target arrays
y = nn_aqi_df["AQI_Range"].values
X = nn_aqi_df.drop("AQI_Range", 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [15]:
print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

Train (1254, 13) (1254,)
Test (418, 13) (418,)


In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  75
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 75)                1050      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                3800      
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 26        
Total params: 6,151
Trainable params: 6,151
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
40/40 [==============================] - 0s 939us/step - loss: 0.7829 - accuracy: 0.8971
Epoch 2/100
40/40 [==============================] - 0s 818us/step - loss: 0.5076 - accuracy: 0.9011
Epoch 3/100
40/40 [==============================] - 0s 818us/step - loss: 0.5014 - accuracy: 0.8979
Epoch 4/100
40/40 [==============================] - 0s 889us/step - loss: 0.4715 - accuracy: 0.8987
Epoch 5/100
40/40 [==============================] - 0s 869us/step - loss: 0.4932 - accuracy: 0.8987
Epoch 6/100
40/40 [==============================] - 0s 864us/step - loss: 0.4721 - accuracy: 0.8987
Epoch 7/100
40/40 [==============================] - 0s 870us/step - loss: 0.4840 - accuracy: 0.8987
Epoch 8/100
40/40 [==============================] - 0s 880us/step - loss: 0.4945 - accuracy: 0.8756
Epoch 9/100
40/40 [==============================] - 0s 818us/step - loss: 0.4809 - accuracy: 0.8939
Epoch 10/100
40/40 [==============================] - 0s 818us/step - loss: 0.4652 - accura

40/40 [==============================] - 0s 895us/step - loss: 0.4372 - accuracy: 0.9027
Epoch 82/100
40/40 [==============================] - 0s 953us/step - loss: 0.4375 - accuracy: 0.9035
Epoch 83/100
40/40 [==============================] - 0s 1ms/step - loss: 0.4328 - accuracy: 0.9043
Epoch 84/100
40/40 [==============================] - 0s 793us/step - loss: 0.4327 - accuracy: 0.9059
Epoch 85/100
40/40 [==============================] - 0s 793us/step - loss: 0.4314 - accuracy: 0.9043
Epoch 86/100
40/40 [==============================] - 0s 814us/step - loss: 0.4292 - accuracy: 0.9035
Epoch 87/100
40/40 [==============================] - 0s 741us/step - loss: 0.4323 - accuracy: 0.9067
Epoch 88/100
40/40 [==============================] - 0s 767us/step - loss: 0.4309 - accuracy: 0.9051
Epoch 89/100
40/40 [==============================] - 0s 716us/step - loss: 0.4332 - accuracy: 0.9011
Epoch 90/100
40/40 [==============================] - 0s 793us/step - loss: 0.4311 - accuracy: 0.

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

14/14 - 0s - loss: 0.6747 - accuracy: 0.9043
Loss: 0.6746677756309509, Accuracy: 0.9043062329292297
